In [0]:
#import dependencies
import numpy as np
import math
import re     #for data cleaning
import time   #to use time model
from google.colab import drive    #for getting all files from drive
import pandas as pd
from bs4 import BeautifulSoup

In [0]:
#imports related to tensor flow
try:
  %tensorflow_version 2.x
except:
  pass
import tensorflow as tf

from tensorflow.keras import layers
import tensorflow_datasets as tfds  

In [0]:
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cols = ["sentiments", "id", "date", "query", "user", "text"]
data = pd.read_csv(
    "/content/drive/My Drive/projects/CNN_for_NLP/data/train.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1" 
)
test_data = pd.read_csv(
    "/content/drive/My Drive/projects/CNN_for_NLP/data/test.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
)

In [0]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

In [0]:
def clean_tweet(tweet):
  tweet = BeautifulSoup(tweet, "lxml").get_text()
  tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
  tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
  tweet = re.sub(r"^[a-zA-Z.!?']", ' ', tweet)
  tweet = re.sub(r" +", " ", tweet)
  return tweet

In [0]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [0]:
data_labels = data.sentiments.values
data_labels[data_labels == 4] = 1

In [0]:
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    data_clean, target_vocab_size=2**16
)
data_inputs = [tokenizer.encode(sentence) for sentence in data_clean]

In [0]:
MAX_LEN = max([len(sentence) for sentence in data_inputs])
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
                                                            value=0,
                                                           padding ="post",
                                                           maxlen = MAX_LEN)

In [0]:
test_idx = np.random.randint(0, 800000, 8000)
test_idx = np.concatenate((test_idx, test_idx+800000))

In [0]:
test_inputs = data_inputs[test_idx]
test_labels = data_labels[test_idx]
train_inputs = np.delete(data_inputs, test_idx, axis=0)
train_labels = np.delete(data_labels, test_idx)

In [0]:
class DCNN(tf.keras.Model):

  def __init__(self,
               vocab_size,
               emb_dim=128,
               nb_filters=50,
               FFN_units=512,
               nb_classes=2,
               dropout_rate=0.1,
               training=False,
               name="dcnn"):
      super(DCNN, self).__init__(name=name)

      self.embedding = layers.Embedding(vocab_size, 
                                        emb_dim)
      self.bigram = layers.Conv1D(filters=nb_filters,
                                kernel_size=2,
                                padding="valid",
                                activation="relu")
      self.pool_1 = layers.GlobalMaxPool1D()
      self.trigram = layers.Conv1D(filters=nb_filters,
                                  kernel_size=3,
                                  padding="valid",
                                  activation="relu")
      self.pool_2 = layers.GlobalMaxPool1D()
      self.fourgram = layers.Conv1D(filters=nb_filters,
                                kernel_size=4,
                                padding="valid",
                                activation="relu")
      self.pool_3 = layers.GlobalMaxPool1D()
      self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
      self.dropout = layers.Dropout(rate=dropout_rate)
      if nb_classes ==2:
        self.last_dense = layers.Dense(units=1,
                                     activation="sigmoid")
      else:
        self.lastdense = layers.Dense(units=nb_classes,
                                    activation = "softmax")
      
  def call (self, inputs, training):
      x = self.embedding(inputs)
      x_1 = self.bigram(x)
      x_1 = self.pool_1(x_1)
      x_2 = self.trigram(x)
      x_2 = self.pool_2(x_2)
      x_3 = self.fourgram(x)
      x_3 = self.pool_3(x_3)

      merged = tf.concat([x_1, x_2, x_3], axis=-1) 
      merged = self.dense_1(merged)
      merged = self.dropout(merged, training)
      output = self.last_dense(merged)

      return output

In [0]:
VOCAB_SIZE = tokenizer.vocab_size

EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = len(set(train_labels))

DROPOUT_RATE = 0.2

BATCH_SIZE = 32
NB_EPOCHS = 5 

In [0]:
Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [0]:
if NB_CLASSES == 2:
  Dcnn.compile(loss="binary_crossentropy",
               optimizer = "adam",
               metrics = ["accuracy"])
else:
  Dcnn.compile(loss="sparse_categorical_crossentropy",
               optimizer="adam",
               metrics=["sparse_categorical_accuracy"])

In [0]:
checkpoint_path = "./drive/My Drive/projects/CNN_for_NLP/ckpt/"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print("Latest Checkpoint Restored!!")

In [0]:
Dcnn.fit(train_inputs,
         train_labels,
         batch_size=BATCH_SIZE,
         epochs=NB_EPOCHS)
ckpt_manager.save()

In [0]:
results = Dcnn.evaluate(test_inputs, test_labels, batch_size=BATCH_SIZE)
print(results)

In [0]:
Dcnn = (np.array([tokenizer.encode("you are funny")]), training=False).numpy()